<a href="https://colab.research.google.com/github/jvwk/Prediction-of-Product-Sales/blob/main/Project_1_Part_5_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 - Part 5 (Core)
###Jaco van Wyk
-----------------------


- Before splitting your data, you can drop duplicates and fix inconsistencies in categorical data.* (*There is a way to do this after the split, but for this project, you may perform this step before the split)
- Identify the features (X) and target (y): Assign the "Item_Outlet_Sales" column as your target and the rest of the relevant variables as your features matrix.
- Perform a train test split
- Create a preprocessing object to prepare the dataset for Machine Learning
- Make sure your imputation of missing values occurs after the train test split using SimpleImputer.

## Data dictionary

Variable | Name	Description
--- | ---
Item_Identifier |	Unique product ID
Item_Weight |	Weight of product
Item_Fat_Content |	Whether the product is low fat or regular
Item_Visibility |	The percentage of total display area of all products in a store allocated to the particular product
Item_Type |	The category to which the product belongs
Item_MRP |	Maximum Retail Price (list price) of the product
Outlet_Identifier |	Unique store ID
Outlet_Establishment_Year |	The year in which store was established
Outlet_Size |	The size of the store in terms of ground area covered
Outlet_Location_Type |	The type of area in which the store is located
Outlet_Type |	Whether the outlet is a grocery store or some sort of supermarket
Item_Outlet_Sales |	Sales of the product in the particular store. This is the target variable to be predicted.

In [126]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

# imports
import pandas as pd
import numpy as np
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

from sklearn import set_config
set_config(transform_output='pandas')

# Load data directly from url
df = pd.read_csv('/content/drive/MyDrive/CodingDojo/02-MachineLearning/Week06/Data/sales_predictions_2023.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [128]:
# Drop duplicates
df.duplicated().sum()


0

No duplicates

In [129]:
# Check for inconsistencies in categorical data
cat_cols = df.select_dtypes("object").columns
for i in cat_cols:
  print(f'Values for {i}:')
  print(df[{i}].value_counts())
  print("\n")

Values for Item_Identifier:
Item_Identifier
FDG33              10
FDW13              10
NCL31               9
FDX04               9
NCQ06               9
                   ..
FDN52               1
FDC23               1
FDT35               1
DRF48               1
FDE52               1
Length: 1559, dtype: int64


Values for Item_Fat_Content:
Item_Fat_Content
Low Fat             5089
Regular             2889
LF                   316
reg                  117
low fat              112
dtype: int64


Values for Item_Type:
Item_Type            
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafo

<ipython-input-129-3dfe81ae880d>:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  print(df[{i}].value_counts())
<ipython-input-129-3dfe81ae880d>:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  print(df[{i}].value_counts())
<ipython-input-129-3dfe81ae880d>:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  print(df[{i}].value_counts())
<ipython-input-129-3dfe81ae880d>:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  print(df[{i}].value_counts())
<ipython-input-129-3dfe81ae880d>:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  print(df[{i}].value_counts())
<ipython-input-129-3dfe81ae880d>:5: FutureWarning: Passing a set as an indexer is deprecated an

In [130]:
# Check for unique categorical feature entries
for i in cat_cols:
  print(f'{i}:\n{df[i].nunique()} ({round(df[i].nunique()/len(df)*100,2)}%) unique entries\n')

Item_Identifier:
1559 (18.29%) unique entries

Item_Fat_Content:
5 (0.06%) unique entries

Item_Type:
16 (0.19%) unique entries

Outlet_Identifier:
10 (0.12%) unique entries

Outlet_Size:
3 (0.04%) unique entries

Outlet_Location_Type:
3 (0.04%) unique entries

Outlet_Type:
4 (0.05%) unique entries



No features with 100% unique entries

Remove categorical features with more than 10 unique values

In [131]:
# Check for constant numerical feature entries
num_cols = df.select_dtypes("number").columns
for i in num_cols:
  print(f'{i}:\nmin = {df[i].min()}\nmax = {df[i].max()}\n')

Item_Weight:
min = 4.555
max = 21.35

Item_Visibility:
min = 0.0
max = 0.328390948

Item_MRP:
min = 31.29
max = 266.8884

Outlet_Establishment_Year:
min = 1985
max = 2009

Item_Outlet_Sales:
min = 33.29
max = 13086.9648



No features with constant entries

In [132]:
# Fix inconsistencies in Item_Fat_Content
df["Item_Fat_Content"] = df["Item_Fat_Content"].replace("LF", "Low Fat")
df["Item_Fat_Content"] = df["Item_Fat_Content"].replace("low fat", "Low Fat")
df["Item_Fat_Content"] = df["Item_Fat_Content"].replace("reg", "Regular")
df["Item_Fat_Content"].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [133]:
# Assign features (X) and target (y)

# The target we are trying to predict
y = df['Item_Outlet_Sales']
# The features we will use to make the prediction
X = df.drop(columns = ['Item_Outlet_Sales'])
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
4776,NCG06,16.350,Low Fat,0.029565,Household,256.4646,OUT018,2009,Medium,Tier 3,Supermarket Type2
7510,FDV57,15.250,Regular,0.000000,Snack Foods,179.7660,OUT018,2009,Medium,Tier 3,Supermarket Type2
5828,FDM27,12.350,Regular,0.158716,Meat,157.2946,OUT049,1999,Medium,Tier 1,Supermarket Type1
5327,FDG24,7.975,Low Fat,0.014628,Baking Goods,82.3250,OUT035,2004,Small,Tier 2,Supermarket Type1
4810,FDD05,19.350,Low Fat,0.016645,Frozen Foods,120.9098,OUT045,2002,NaN,Tier 2,Supermarket Type1


In [134]:
# Create a preprocessing object to prepare the dataset for Machine Learning

# Create lists of column names for numeric, categorical and ordinal columns
num_cols = num_cols.drop(labels = 'Item_Outlet_Sales')
cat_cols = ['Outlet_Identifier', 'Outlet_Type']
ord_cols = ['Item_Fat_Content', 'Outlet_Size', 'Outlet_Location_Type']


In [135]:
# Instantiate numerical preprocessors and pipeline
impute_median = SimpleImputer(strategy='median')
scaler_num = StandardScaler()
num_pipe = make_pipeline(impute_median, scaler_num)
num_pipe


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [136]:
# Instantiate nominal preprocessors and pipeline
impute_missing = SimpleImputer(strategy='constant', fill_value = "MISSING")
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_pipe = make_pipeline(impute_missing, ohe_encoder)
ohe_pipe


Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='MISSING', strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [137]:
# Specifying order of categories for our  Ordinal Qual/Cond Columms
fat_content_order = ['Low Fat','Regular']
outlet_size_order = ['Small','Medium', 'High']
outlet_loc_type_order = ['Tier 1','Tier 2', 'Tier 3']
# Making the list of order lists for OrdinalEncoder
ordinal_category_orders = [fat_content_order, outlet_size_order, outlet_loc_type_order]

# Instantiate ordinal preprocessors and pipeline
impute_mostfreq = SimpleImputer(strategy='most_frequent')
ord_encoder = OrdinalEncoder(categories=ordinal_category_orders)
scaler_ord = StandardScaler()
ord_pipe = make_pipeline(impute_mostfreq, ord_encoder, scaler_ord)
ord_pipe


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['Low Fat', 'Regular'],
                                            ['Small', 'Medium', 'High'],
                                            ['Tier 1', 'Tier 2', 'Tier 3']])),
                ('standardscaler', StandardScaler())])

In [138]:
# Making a numeric tuple for ColumnTransformer
num_tuple = ('numeric', num_pipe, num_cols)
num_tuple


('numeric',
 Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                 ('standardscaler', StandardScaler())]),
 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
        'Outlet_Establishment_Year'],
       dtype='object'))

In [139]:
# Making a ohe_tuple for ColumnTransformer
ohe_tuple = ('categorical', ohe_pipe, cat_cols)
ohe_tuple


('categorical',
 Pipeline(steps=[('simpleimputer',
                  SimpleImputer(fill_value='MISSING', strategy='constant')),
                 ('onehotencoder',
                  OneHotEncoder(handle_unknown='ignore', sparse_output=False))]),
 ['Outlet_Identifier', 'Outlet_Type'])

In [140]:
# Making an ordinal tuple for ColumnTransformer
ord_tuple = ('ordinal', ord_pipe, ord_cols)
ord_tuple


('ordinal',
 Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                 ('ordinalencoder',
                  OrdinalEncoder(categories=[['Low Fat', 'Regular'],
                                             ['Small', 'Medium', 'High'],
                                             ['Tier 1', 'Tier 2', 'Tier 3']])),
                 ('standardscaler', StandardScaler())]),
 ['Item_Fat_Content', 'Outlet_Size', 'Outlet_Location_Type'])

In [141]:
# Use the tuples to create a ColumnTransformer to preprocess the data; set verbose_feature_names_out to False
col_transformer = ColumnTransformer([num_tuple, ohe_tuple, ord_tuple],
                                    verbose_feature_names_out=False)
col_transformer

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')),
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='MISSING',
                                                                strategy='constant')),
                                                 (...
                                 ['Outlet_Identifier', 'Outlet_Type']),
                                ('ordinal',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Low '
                                                                              'Fat',
                                                                              'Regular'],
                                                                             ['Small',
                                                                              'Medium',
                                                                              'High'],
                                                                             ['Tier '
                                                                              '1',
                                                                              'Tier '
                                                                              '2',
                                                                              'Tier '
                                                                              '3']])),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Item_Fat_Content', 'Outlet_Size',
                                  'Outlet_Location_Type'])],
                  verbose_feature_names_out=False)

In [142]:
# Fit on training data
col_transformer.fit(X_train)

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')),
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='MISSING',
                                                                strategy='constant')),
                                                 (...
                                 ['Outlet_Identifier', 'Outlet_Type']),
                                ('ordinal',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Low '
                                                                              'Fat',
                                                                              'Regular'],
                                                                             ['Small',
                                                                              'Medium',
                                                                              'High'],
                                                                             ['Tier '
                                                                              '1',
                                                                              'Tier '
                                                                              '2',
                                                                              'Tier '
                                                                              '3']])),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Item_Fat_Content', 'Outlet_Size',
                                  'Outlet_Location_Type'])],
                  verbose_feature_names_out=False)

In [143]:
# Transform the training data
X_train_processed = col_transformer.transform(X_train)
# Transform the testing data
X_test_processed = col_transformer.transform(X_test)

In [144]:
X_train_processed.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Identifier_OUT010,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,...,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Fat_Content,Outlet_Size,Outlet_Location_Type
4776,0.827485,-0.712775,1.828109,1.327849,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.740321,0.287374,1.084948
7510,0.566644,-1.291052,0.603369,1.327849,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.350766,0.287374,1.084948
5828,-0.121028,1.813319,0.244541,0.136187,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.350766,0.287374,-1.384777
5327,-1.158464,-1.004931,-0.952591,0.732018,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.740321,-1.384048,-0.149914
4810,1.538870,-0.965484,-0.336460,0.493686,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.740321,0.287374,-0.149914


In [145]:
# Check processed training data types are numeric (except passed through features)
X_train_processed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6392 entries, 4776 to 7270
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Item_Weight                    6392 non-null   float64
 1   Item_Visibility                6392 non-null   float64
 2   Item_MRP                       6392 non-null   float64
 3   Outlet_Establishment_Year      6392 non-null   float64
 4   Outlet_Identifier_OUT010       6392 non-null   float64
 5   Outlet_Identifier_OUT013       6392 non-null   float64
 6   Outlet_Identifier_OUT017       6392 non-null   float64
 7   Outlet_Identifier_OUT018       6392 non-null   float64
 8   Outlet_Identifier_OUT019       6392 non-null   float64
 9   Outlet_Identifier_OUT027       6392 non-null   float64
 10  Outlet_Identifier_OUT035       6392 non-null   float64
 11  Outlet_Identifier_OUT045       6392 non-null   float64
 12  Outlet_Identifier_OUT046       6392 non-null 

In [146]:
# Check scaling: mean close to 0 and st dev close to 1
X_train_processed[num_cols].describe().loc[['mean', 'std']].round(5)

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
mean,-0.00000,-0.00000,0.00000,-0.00000
std,1.00008,1.00008,1.00008,1.00008
